In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../src'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/enis/projects/nna/src


In [10]:
from datetime import timedelta
import datetime 
import os

from fileUtils import read_file_properties_v2,find_filesv2,list_files,get_labeled_exif
from fileUtils import get_audio,str2timestamp,query_audio

import pandas as pd
from pathlib import Path

from sklearn.neighbors import KDTree
import numpy as np

In [ ]:
list_files("/scratch/enis/data/nna/real_2organize/")

array([[0, 1],
       [1, 0],
       [2, 1],
       [3, 4],
       [4, 3],
       [5, 4]])

In [ ]:
output = find_filesv2(location,start_time,end_time,length,0,file_properties_df)
sorted_filtered,start_time,end_time,start_time_org,end_time_org = output

In [4]:
p_path="/home/enis/projects/nna/data/cameraTraps/test.txt"

with open(p_path) as p_file:
    photo_paths=p_file.readlines()
    photo_paths = [i.strip() for i in photo_paths]

other_folder=["anwr_35_2019_101RECNX_RCNX3373.JPG",
"anwr_42_2019_100RECNX_RCNX3574.JPG",
"ivvavik_SINP03_2019_100RECNX_IMG_3219.JPG",
"ivvavik_SINP05_2019_100RECNX_IMG_2867.JPG",
"ivvavik_SINP06_2019_100RECNX_IMG_1884.JPG",
"ivvavik_SINP09_2019_100RECNX_IMG_2743.JPG",
"prudhoe_17_2019_100RECNX_RCNX3916.JPG",]

parent_path1="/tank/data/nna/examples/randomPhotos10k/"
parent_path2="/tank/data/nna/examples/randomPhotos1k/"

file_properties_df=pd.read_pickle("../realdata_v2No_stinchcomb.pkl")

In [7]:
# photo with Caribou
photo_paths=['anwr_37_2019_100RECNX_RCNX9317.jpg']

In [12]:
for img_file_name in photo_paths:

    img_file_name=(img_file_name)[:-4] + ".JPG"
    if img_file_name in other_folder:
        parent_path = parent_path2
    else:
        parent_path = parent_path1
    location=img_file_name.split("_")[1]
    
    labeled = get_labeled_exif(parent_path+img_file_name)
    # print(labeled)

    timestamp=labeled["DateTimeDigitized"]
    timestamp=datetime.datetime.strptime(timestamp, '%Y:%m:%d %H:%M:%S')

    
    start_time=timestamp - timedelta(seconds=30)
    start_time=start_time.strftime('%Y-%m-%d_%H:%M:%S')
    length=60 # seconds of audio, if set end_time is IGNORED
    end_time=''  # or datetime object
    print(img_file_name,start_time)

    file_name=img_file_name[:-4]

    buffer=10800 # how much buffer to each side

    tmp_folder="../trap_photo_audio_new3/"
    print(start_time)
    output = find_filesv2(location,start_time,end_time,length,0,file_properties_df)
    sorted_filtered,start_time,end_time,start_time_org,end_time_org = output

anwr_37_2019_100RECNX_RCNX9317.JPG 2019-06-05_09:21:06
2019-06-05_09:21:06


In [14]:
sorted_filtered

,site_id,locationId,site_name,recorderId,hour_min_sec,year,month,day,region,timestamp,durationSec,timestampEnd
/tank/data/nna/real/anwr/37/2019/S4A10279_20190605_091602.flac,37,37,,S4A10279,091602,2019,06,05,anwr,2019-06-05 09:16:02,4438,2019-06-05 10:30:00


In [16]:
!find /scratch/enis/data/nna/real_2organize/ -iname "S4A10279_20190605_091602*"

/scratch/enis/data/nna/real_2organize/37B/S4A10279_20190605_091602_vgg
/scratch/enis/data/nna/real_2organize/37B/S4A10279_20190605_091602_vgg/S4A10279_20190605_091602_embeddings000.npy
/scratch/enis/data/nna/real_2organize/37B/S4A10279_20190605_091602_vgg/S4A10279_20190605_091602_rawembeddings000.npy
/scratch/enis/data/nna/real_2organize/37B/S4A10279_20190605_091602_preprocessed
/scratch/enis/data/nna/real_2organize/37B/S4A10279_20190605_091602_FCmodel
/scratch/enis/data/nna/real_2organize/37B/S4A10279_20190605_091602_FCmodel/S4A10279_20190605_091602_FCmodel000.npy


In [17]:
raw_embed=np.load("/scratch/enis/data/nna/real_2organize/37B/S4A10279_20190605_091602_vgg/S4A10279_20190605_091602_rawembeddings000.npy")

In [25]:
start_seconds=(start_time-sorted_filtered["timestamp"])[0].total_seconds()

304.0

In [28]:
raw_embed_audio=raw_embed[int(start_seconds):int(start_seconds)+60]

In [29]:
raw_embed_audio.shape

(60, 128)

In [35]:
all_embeddings=list_files("/scratch/enis/data/nna/real_2organize/37B/")

In [37]:
all_embeddings=[i for i in all_embeddings if "_embeddings" in i]

In [39]:
embeds=[]
for embed_file in all_embeddings:
    embed=np.load(embed_file)
    embeds.append(embed[:])

In [42]:
concat_embeds=np.concatenate(embeds)

In [43]:
concat_embeds.shape

(350922, 128)

In [46]:
# X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
kdt = KDTree(concat_embeds, leaf_size=30, metric='euclidean')


In [53]:
kdt.query(raw_embed_audio[0:20,:], k=3, return_distance=True)

(array([[1494.67256952, 1502.55459562, 1505.98213109],
        [1494.75332064, 1502.57033279, 1506.00850629],
        [1494.7590155 , 1502.62885548, 1506.0992402 ],
        [1494.75480084, 1502.62485893, 1506.06302295],
        [1494.83601786, 1502.58640697, 1506.06398779],
        [1494.69913671, 1502.53369709, 1505.94704574],
        [1494.65535268, 1502.52139964, 1505.93703425],
        [1494.74797331, 1502.5865798 , 1506.05499663],
        [1494.6887206 , 1502.52608752, 1505.9391115 ],
        [1494.75438336, 1502.52652462, 1505.98018084],
        [1494.76955244, 1502.59520893, 1506.03198442],
        [1494.55283706, 1502.3360394 , 1505.76089385],
        [1494.76322255, 1502.56659498, 1506.05070249],
        [1494.58712182, 1502.41450198, 1505.78328712],
        [1494.78750892, 1502.54519469, 1506.01676386],
        [1494.68973819, 1502.50927576, 1505.93545869],
        [1494.64909438, 1502.41045771, 1506.01768248],
        [1494.59302478, 1502.40586575, 1506.01895974],
        [1

In [58]:
count=0
for i,em in enumerate(embeds):
    count+=(em.shape[0])
    if count>=305326:
        print(i,count)

67 305810
68 310248
69 314808
70 319368
71 323928
72 328488
73 333048
74 337486
75 341924
76 346484
77 350922


In [62]:
count=0
for i,em in enumerate(embeds):
    if i==67:
        print(count,em.shape[0])
    count+=(em.shape[0])

        

301372 4438


In [60]:
(305326-301372)

3954

In [63]:
all_embeddings[67]

'/scratch/enis/data/nna/real_2organize/37B/S4A10279_20190603_194602_vgg/S4A10279_20190603_194602_embeddings000.npy'

In [66]:
3954//60,3954%60,

(65, 54)

In [74]:
!mp3splt -d "./" -f -D /tank/data/nna/real/anwr/37/2019/S4A10279_20190603_194602.flac 50.54 51.54

mp3splt 2.4.2 (13/05/12) - using libmp3splt 0.7.2
	Matteo Trotta <mtrotta AT users.sourceforge.net>
	Alexandru Munteanu <io_fx AT yahoo.fr>
THIS SOFTWARE COMES WITH ABSOLUTELY NO WARRANTY! USE AT YOUR OWN RISK!

Searching for plugins ...
Scanning plugins in the directory _/usr/lib/libmp3splt_
Looking at the file _libsplt_mp3.so.0.0.0_

Looking at the file _libsplt_ogg.so.0_

Looking at the file _libsplt_ogg.so.0.0.0_

Looking at the file _libsplt_mp3.so.0_

Checking if _/usr/lib/libmp3splt/libsplt_mp3.so.0_ is like _/usr/lib/libmp3splt/libsplt_ogg.so.0_ 
Scanning plugins in the directory _/home/enis/.libmp3splt_
Scanning plugins in the directory _./_

Trying to open the plugin _/usr/lib/libmp3splt/libsplt_ogg.so.0_ ...
 - success !

Trying to open the plugin _/usr/lib/libmp3splt/libsplt_mp3.so.0_ ...
 - success !

Number of plugins found: _2_
plugin filename = _/usr/lib/libmp3splt/libsplt_ogg.so.0_
plugin name = _ogg vorbis (libvorbis)_
plugin version = _1.000000_
extension = _.ogg_

p

In [68]:
!ls/tank/data/nna/real/anwr/37/2019/S4A10279_20190603_194602.flac

books_read.png			get_audio4photos.ipynb
cum_hist_1000bins_025limit.svg	listen_photos.ipynb
cum_hist_1000bins.png		listen_segments_backup.ipynb
cum_hist_1000bins.svg		listen_segments.ipynb
cumulative_histogram2.svg	nearest_neighbors.ipynb
cumulative_histogram.jpg	Untitled1.ipynb
cumulative_histogram.png	Untitled.ipynb
